## Project 6

In [1]:
import os
import subprocess
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import scipy
import requests
from imdbpie import Imdb
import nltk
import matplotlib.pyplot as plt
import urllib2
from bs4 import BeautifulSoup
import nltk
import collections
import re
import csv
import psycopg2

%matplotlib inline

### Pre-Work: Write a problem statement 

## Part 1: Acquire the Data

#### 1. Connect to the IMDB API

In [2]:
imdb = Imdb()
imdb = Imdb(anonymize=True)

#### 2. Query the top 250 rated movies in the database

In [3]:
imdb.top_250()

[{u'can_rate': True,
  u'image': {u'height': 1388,
   u'url': u'https://images-na.ssl-images-amazon.com/images/M/MV5BODU4MjU4NjIwNl5BMl5BanBnXkFtZTgwMDU2MjEyMDE@._V1_.jpg',
   u'width': 933},
  u'num_votes': 1718172,
  u'rating': 9.3,
  u'tconst': u'tt0111161',
  u'title': u'The Shawshank Redemption',
  u'type': u'feature',
  u'year': u'1994'},
 {u'can_rate': True,
  u'image': {u'height': 1129,
   u'url': u'https://images-na.ssl-images-amazon.com/images/M/MV5BNTUxOTdjMDMtMWY1MC00MjkxLTgxYTMtYTM1MjU5ZTJlNTZjXkEyXkFqcGdeQXVyNTA4NzY1MzY@._V1_.jpg',
   u'width': 798},
  u'num_votes': 1174396,
  u'rating': 9.2,
  u'tconst': u'tt0068646',
  u'title': u'The Godfather',
  u'type': u'feature',
  u'year': u'1972'},
 {u'can_rate': True,
  u'image': {u'height': 1140,
   u'url': u'https://images-na.ssl-images-amazon.com/images/M/MV5BNDVjZjgxNTgtMGNhMC00YWU0LTg0YTQtNTkxNzBjMDBkNWYyXkEyXkFqcGdeQXVyNTA4NzY1MzY@._V1_.jpg',
   u'width': 800},
  u'num_votes': 804768,
  u'rating': 9,
  u'tconst': u'tt0071

In [4]:
#Delete all useless columns 
movies = []
for dct in imdb.top_250():
    del dct['image']
    del dct['can_rate']
    del dct['type']
    movies.append(dct)

In [5]:
Vote = []
Rate = []
ID = []
Year = []
Title = []
for dct in movies:
    Vote.append(dct['num_votes'])
    Rate.append(dct['rating'])
    ID.append(dct['tconst'])
    Year.append(dct['year'])
    Title.append(dct['title'])

In [6]:
df = pd.DataFrame({
        "ID": ID,
        'Title': Title,
        'Year': Year,
        'Vote': Vote,
        'Rate': Rate
    })

In [7]:
df

ID  Rate                                              Title  \
0    tt0111161   9.3                           The Shawshank Redemption   
1    tt0068646   9.2                                      The Godfather   
2    tt0071562   9.0                             The Godfather: Part II   
3    tt0468569   9.0                                    The Dark Knight   
4    tt0050083   8.9                                       12 Angry Men   
5    tt0108052   8.9                                   Schindler's List   
6    tt0110912   8.9                                       Pulp Fiction   
7    tt0167260   8.9      The Lord of the Rings: The Return of the King   
8    tt0060196   8.9                     The Good, the Bad and the Ugly   
9    tt0137523   8.8                                         Fight Club   
10   tt0120737   8.8  The Lord of the Rings: The Fellowship of the Ring   
11   tt0080684   8.8     Star Wars: Episode V - The Empire Strikes Back   
12   tt0109830   8.8                                       Forrest Gump   
13   tt1375666   8.8                                          Inception   
14   tt0167261   8.7              The Lord of the Rings: The Two Towers   
15   tt0073486   8.7                    One Flew Over the Cuckoo's Nest   
16   tt0099685   8.7                                         Goodfellas   
17   tt0133093   8.7                                         The Matrix   
18   tt0047478   8.7                                      Seven Samurai   
19   tt0076759   8.7                 Star Wars: Episode IV - A New Hope   
20   tt0317248   8.7                                        City of God   
21   tt0114369   8.6                                              Se7en   
22   tt0102926   8.6                           The Silence of the Lambs   
23   tt0038650   8.6                              It's a Wonderful Life   
24   tt0114814   8.6                                 The Usual Suspects   
25   tt0118799   8.6                                  Life Is Beautiful   
26   tt0110413   8.6                             Léon: The Professional   
27   tt0245429   8.6                                      Spirited Away   
28   tt0120815   8.6                                Saving Private Ryan   
29   tt0064116   8.6                       Once Upon a Time in the West   
..         ...   ...                                                ...   
220  tt0073195   8.0                                               Jaws   
221  tt0036868   8.1                        The Best Years of Our Lives   
222  tt0338564   8.1                                   Infernal Affairs   
223  tt0113247   8.1                                           La Haine   
224  tt0072684   8.1                                       Barry Lyndon   
225  tt0044079   8.1                               Strangers on a Train   
226  tt0264464   8.0                                Catch Me If You Can   
227  tt0401792   8.1                                           Sin City   
228  tt0058946   8.1                              The Battle of Algiers   
229  tt0072890   8.0                                  Dog Day Afternoon   
230  tt0041546   8.1                           Kind Hearts and Coronets   
231  tt1865505   8.2                                    Song of the Sea   
232  tt1663202   8.1                                       The Revenant   
233  tt1431045   8.1                                           Deadpool   
234  tt0325980   8.1  Pirates of the Caribbean: The Curse of the Bla...   
235  tt1454029   8.1                                           The Help   
236  tt1954470   8.3                                 Gangs of Wasseypur   
237  tt2084970   8.1                                 The Imitation Game   
238  tt1220719   8.1                                             Ip Man   
239  tt3659388   8.0                                        The Martian   
240  tt0046250   8.1                                      Roman Holiday   
241  tt0058461   8.0                 

#### 3. Only select the top 25 movies and delete the uncessary rows

In [8]:
df_top_25 = df.head(25)

#### 4. Write the Results to a csv

In [9]:
df_top_25.to_csv('top25.csv')

## Part 2: Wrangle the text data

#### 1. Convert the listing identification numbers (tconst) from the first dataframe to a list

In [10]:
id_list = list(df_top_25.ID)

In [11]:
type(id_list)

list

#### 2. Scrape the reviews for the top 25 movies

*Hint*: Use a loop to scrape each page at once

In [13]:
df = pd.read_csv('top25.csv')
id_list = list(df.ID)
df

Unnamed: 0         ID  Rate  \
0            0  tt0111161   9.3   
1            1  tt0068646   9.2   
2            2  tt0071562   9.0   
3            3  tt0468569   9.0   
4            4  tt0050083   8.9   
5            5  tt0108052   8.9   
6            6  tt0110912   8.9   
7            7  tt0167260   8.9   
8            8  tt0060196   8.9   
9            9  tt0137523   8.8   
10          10  tt0120737   8.8   
11          11  tt0080684   8.8   
12          12  tt0109830   8.8   
13          13  tt1375666   8.8   
14          14  tt0167261   8.7   
15          15  tt0073486   8.7   
16          16  tt0099685   8.7   
17          17  tt0133093   8.7   
18          18  tt0047478   8.7   
19          19  tt0076759   8.7   
20          20  tt0317248   8.7   
21          21  tt0114369   8.6   
22          22  tt0102926   8.6   
23          23  tt0038650   8.6   
24          24  tt0114814   8.6   

                                                Title     Vote  Year  
0                            The Shawshank Redemption  1718172  1994  
1                                       The Godfather  1174396  1972  
2                              The Godfather: Part II   804768  1974  
3                                     The Dark Knight  1704360  2008  
4                                        12 Angry Men   457455  1957  
5                                    Schindler's List   879846  1993  
6                                        Pulp Fiction  1346128  1994  
7       The Lord of the Rings: The Return of the King  1234671  2003  
8                      The Good, the Bad and the Ugly   511481  1966  
9                                          Fight Club  1371330  1999  
10  The Lord of the Rings: The Fellowship of the Ring  1257328  2001  
11     Star Wars: Episode V - The Empire Strikes Back   852316  1980  
12                                       Forrest Gump  1275914  1994  
13                                          Inception  1494900  2010  
14              The Lord of the Rings: The Two Towers  1117984  2002  
15                    One Flew Over the Cuckoo's Nest   690376  1975  
16                                         Goodfellas   740985  1990  
17                                         The Matrix  1237553  1999  
18                                      Seven Samurai   232908  1954  
19                 Star Wars: Episode IV - A New Hope   925378  1977  
20                                        City of God   541304  2002  
21                                              Se7en  1042588  1995  
22                           The Silence of the Lambs   904030  1991  
23                              It's a Wonderful Life   280095  1946  
24                                 The Usual Suspects   753063  1995

In [14]:
# Three pages reviews for each movie
Reviews = []
for i in id_list:
    for p in [0,10,20]:
        url = 'http://www.imdb.com/title/' + i + '/' + 'reviews?start=' + str(p) 
        contents = BeautifulSoup(urllib2.urlopen(url),"lxml")
        content = contents.find('div',attrs={'id':'tn15content'})
        review = content.find_all('p')
        for rew in review:
            Reviews.append(rew.text)  

In [15]:
len(Reviews)

1074

In [16]:
w = []
sentences = nltk.sent_tokenize(str(Reviews))
for s in sentences:
    words = nltk.word_tokenize(str(s))
    tagged = nltk.pos_tag(words)
    for (word, part_of_speech) in tagged:
        if part_of_speech == 'JJ' or part_of_speech == 'JJS':
            w.append(word)

"""
w = []
sentences = nltk.sent_tokenize(str(Reviews))
for s in sentences:
    words = nltk.word_tokenize(str(s))
    tagged = nltk.pos_tag(words)
    for (word, part_of_speech) in tagged:
        if word == 'extraordinary':
            print word, part_of_speech
"""



"""
count = 0
for i in review_1:
    if i == 'extraordinary':
        count+=1
    
print count

"""

In [17]:
from collections import Counter
dct = Counter(w)

In [18]:
for key, value in sorted(dct.iteritems(), key=lambda (k,v): (v,k)):
    print "%s: %s" % (key, value)

'10: 1
'12: 1
'Apocalypse: 1
'Apt: 1
'Bullet: 1
'Fight: 1
'Last: 1
'Reservoir: 1
'Un: 1
'X-Men'.\nFor: 1
'\nright: 1
'big: 1
'civilian: 1
'intelligent: 1
'old: 1
'ooh'-ed: 1
'ooh'-ed\nor: 1
'open: 1
'top: 1
'ultra-modern: 1
'unfilmable: 1
'wow: 1
*: 1
****: 1
********ABSOLUTE: 1
*****END: 1
**SPOILER**The: 1
*John: 1
*Mean: 1
*THE*: 1
*any*: 1
*con*: 1
*only*: 1
*sure*: 1
*very*: 1
-Tom: 1
-\nby: 1
-\nexpert: 1
-\nor: 1
-type: 1
.Forrest: 1
.This: 1
.\nThis: 1
.a\nBatman: 1
.in: 1
/: 1
//www.aqweeb.com/2016/05/hackers-Movies-P2.html: 1
007-like: 1
1-hour: 1
10-11: 1
12-: 1
12-year-old: 1
120-: 1
15-page: 1
16th: 1
17th: 1
1928-1946: 1
2-1/2: 1
2-1/2-: 1
2-hour: 1
20-minute: 1
22nd: 1
25\xa2: 1
3-D: 1
3-hour: 1
3-hour-long: 1
3-way: 1
30-40: 1
30-hour: 1
30-minute: 1
30-year\nstretch: 1
35-40: 1
3\ntheater-release: 1
4-5: 1
4-6: 1
4-hour: 1
4/4/1979-1/22/2008.\n: 1
50-year-old: 1
6\'5: 1
7-9: 1
9-5: 1
9-book: 1
9-like: 1
AMAZING: 1
ANGRY: 1
AOK.\r\n\r\nGary: 1
AT-AT: 1
AWFUL\nEnglish: 1

In [19]:
Review = []

for s in Reviews:
    if s != "*** This review may contain spoilers ***" and s != 'Add another review':
        Review.append(s)
        

In [20]:
len(Review)

750

In [21]:
review_1 =Review[0:30]
review_2 =Review[30:60]
review_3 =Review[60:90]
review_4 =Review[90:120]
review_5 =Review[120:150]
review_6 =Review[150:180]
review_7 =Review[180:210]
review_8 =Review[210:240]
review_9 =Review[240:270]
review_10 =Review[270:300]
review_11 =Review[300:330]
review_12 =Review[330:360]
review_13 =Review[360:390]
review_14 =Review[390:420]
review_15 =Review[420:450]
review_16 =Review[450:480]
review_17 =Review[480:510]
review_18 =Review[510:540]
review_19 =Review[540:570]
review_20 =Review[570:600]
review_21 =Review[600:630]
review_22 =Review[630:660]
review_23 =Review[660:690]
review_24 =Review[690:720]
review_25 =Review[720:750] 

review_list = [review_1,review_2,review_3,review_4,review_5,review_6,review_7,
              review_8,review_9,review_10,review_11,review_12,review_13,review_14,
              review_15,review_16,review_17,review_18,review_19,review_20,review_21,
              review_22,review_23,review_24,review_25]


####  7. Find the rows with the top five descriptive words

In [13]:
# Great  good   best   greatest   perfect

In [22]:
great = []
good = []
best = []
greatest = []
perfect = []
for review in review_list:
    sum_of_great=0
    sum_of_good=0
    sum_of_best=0
    sum_of_greatest=0
    sum_of_perfect=0
    words = nltk.word_tokenize(str(review))
    for word in words:
        if word == 'great':
            sum_of_great+=1
        elif word == 'good':
            sum_of_good+=1
        elif word == 'best':
            sum_of_best+=1
        elif word == 'greatest':
            sum_of_greatest+=1
        elif word == 'perfect':
            sum_of_perfect+=1
    great.append(sum_of_great)
    good.append(sum_of_good)
    best.append(sum_of_best)
    greatest.append(sum_of_greatest)
    perfect.append(sum_of_perfect)

In [23]:
len(great)

25

In [24]:
words_freq = pd.DataFrame({'ID':df.ID,
                          'Great': great,
                          'Good': good,
                          'Best': best,
                          'Greatest':greatest,
                          'Perfect':perfect})

In [25]:
words_freq

Best  Good  Great  Greatest         ID  Perfect
0     29    22     21         5  tt0111161        3
1     20    21     29        15  tt0068646       13
2     19    22     22         6  tt0071562        8
3     31    34     22         1  tt0468569        5
4     13     9     27         5  tt0050083        6
5     13    17      6         4  tt0108052        4
6     15     9      9         4  tt0110912        0
7     27    22     27        11  tt0167260        2
8     31    27     24        10  tt0060196        3
9     19    12     23         2  tt0137523        4
10     8    36     30         9  tt0120737       11
11    24    15     25         7  tt0080684        2
12    18    10     18         2  tt0109830        3
13    12    18     16         2  tt1375666        4
14    17    13     17         6  tt0167261        1
15    17    19     20         8  tt0073486        6
16    23    13     21         7  tt0099685        5
17    13    10     20         0  tt0133093        3
18     6     6     26         5  tt0047478        5
19    16    11     25        12  tt0076759        3
20     7    12      8         1  tt0317248        6
21    18    20     15         6  tt0114369        6
22    20    11      9         2  tt0102926        6
23    14    19     16         4  tt0038650        3
24     9    14     11         8  tt0114814        3

#### 8. Write the results to a csv

In [26]:
words_freq.to_csv('words.csv')

## Part 3: Combine Tables in PostgreSQL

#### 1. Import your two .csv data files into your Postgre Database as two different tables

For ease, we can call these table1 and table2

In [27]:
%load_ext sql

//anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
//anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [28]:
%%sql postgresql://tianjiaoxie@localhost:5432
SELECT * FROM information_schema.tables LIMIT 3;

3 rows affected.


[(u'tianjiaoxie', u'pg_catalog', u'pg_statistic', u'BASE TABLE', None, None, None, None, None, u'YES', u'NO', None),
 (u'tianjiaoxie', u'pg_catalog', u'pg_type', u'BASE TABLE', None, None, None, None, None, u'YES', u'NO', None),
 (u'tianjiaoxie', u'pg_catalog', u'pg_authid', u'BASE TABLE', None, None, None, None, None, u'YES', u'NO', None)]

In [29]:
%%sql
CREATE DATABASE IMDB;

(psycopg2.ProgrammingError) database "imdb" already exists
 [SQL: 'CREATE DATABASE IMDB;']


In [30]:
%%sql postgresql://tianjiaoxie@localhost:5432/imdb
SELECT * FROM information_schema.tables LIMIT 3;

3 rows affected.


[(u'imdb', u'pg_catalog', u'pg_statistic', u'BASE TABLE', None, None, None, None, None, u'YES', u'NO', None),
 (u'imdb', u'pg_catalog', u'pg_type', u'BASE TABLE', None, None, None, None, None, u'YES', u'NO', None),
 (u'imdb', u'pg_catalog', u'pg_authid', u'BASE TABLE', None, None, None, None, None, u'YES', u'NO', None)]

In [ ]:
# Tried below method to upload CSV onto database but failed

%%sql
DROP TABLE IF EXISTS top_25;
CREATE TABLE top_25
(ID varchar,
 Rate FLOAT,
 Title varchar,
 Vote integer,
 Year integer);
COPY top_25 FROM 'Top25_movies.csv' DELIMITER ',' CSV HEADER;

%%sql
DROP TABLE IF EXISTS words;
CREATE TABLE words
(
 Best integer,
 Good integer,
 Great integer,
 Greatest integer,
 ID varchar,
 Perfect integer);

COPY words FROM 'Five_Descriptive_words.csv' DELIMITER ',' CSV HEADER;

In [ ]:
# Use below method to upload

In [31]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://tianjiaoxie@localhost:5432/imdb')
pd.read_sql("SELECT * FROM information_schema.tables LIMIT 3;", engine)

table_catalog table_schema    table_name  table_type  \
0          imdb   pg_catalog  pg_statistic  BASE TABLE   
1          imdb   pg_catalog       pg_type  BASE TABLE   
2          imdb   pg_catalog     pg_authid  BASE TABLE   

  self_referencing_column_name reference_generation user_defined_type_catalog  \
0                         None                 None                      None   
1                         None                 None                      None   
2                         None                 None                      None   

  user_defined_type_schema user_defined_type_name is_insertable_into is_typed  \
0                     None                   None                YES       NO   
1                     None                   None                YES       NO   
2                     None                   None                YES       NO   

  commit_action  
0          None  
1          None  
2          None

In [32]:
df = pd.read_csv('words.csv')

In [33]:
df.to_sql('words',con=engine)

In [34]:
pd.read_sql('SELECT * FROM words', engine)

index  Unnamed: 0  Best  Good  Great  Greatest         ID  Perfect
0       0           0    29    22     21         5  tt0111161        3
1       1           1    20    21     29        15  tt0068646       13
2       2           2    19    22     22         6  tt0071562        8
3       3           3    31    34     22         1  tt0468569        5
4       4           4    13     9     27         5  tt0050083        6
5       5           5    13    17      6         4  tt0108052        4
6       6           6    15     9      9         4  tt0110912        0
7       7           7    27    22     27        11  tt0167260        2
8       8           8    31    27     24        10  tt0060196        3
9       9           9    19    12     23         2  tt0137523        4
10     10          10     8    36     30         9  tt0120737       11
11     11          11    24    15     25         7  tt0080684        2
12     12          12    18    10     18         2  tt0109830        3
13     13          13    12    18     16         2  tt1375666        4
14     14          14    17    13     17         6  tt0167261        1
15     15          15    17    19     20         8  tt0073486        6
16     16          16    23    13     21         7  tt0099685        5
17     17          17    13    10     20         0  tt0133093        3
18     18          18     6     6     26         5  tt0047478        5
19     19          19    16    11     25        12  tt0076759        3
20     20          20     7    12      8         1  tt0317248        6
21     21          21    18    20     15         6  tt0114369        6
22     22          22    20    11      9         2  tt0102926        6
23     23          23    14    19     16         4  tt0038650        3
24     24          24     9    14     11         8  tt0114814        3

In [35]:
df1 = pd.read_csv('top25.csv')

In [36]:
df1.to_sql('top25',con=engine)

In [37]:
pd.read_sql('SELECT * FROM top25',engine)

index  Unnamed: 0         ID  Rate  \
0       0           0  tt0111161   9.3   
1       1           1  tt0068646   9.2   
2       2           2  tt0071562   9.0   
3       3           3  tt0468569   9.0   
4       4           4  tt0050083   8.9   
5       5           5  tt0108052   8.9   
6       6           6  tt0110912   8.9   
7       7           7  tt0167260   8.9   
8       8           8  tt0060196   8.9   
9       9           9  tt0137523   8.8   
10     10          10  tt0120737   8.8   
11     11          11  tt0080684   8.8   
12     12          12  tt0109830   8.8   
13     13          13  tt1375666   8.8   
14     14          14  tt0167261   8.7   
15     15          15  tt0073486   8.7   
16     16          16  tt0099685   8.7   
17     17          17  tt0133093   8.7   
18     18          18  tt0047478   8.7   
19     19          19  tt0076759   8.7   
20     20          20  tt0317248   8.7   
21     21          21  tt0114369   8.6   
22     22          22  tt0102926   8.6   
23     23          23  tt0038650   8.6   
24     24          24  tt0114814   8.6   

                                                Title     Vote  Year  
0                            The Shawshank Redemption  1718172  1994  
1                                       The Godfather  1174396  1972  
2                              The Godfather: Part II   804768  1974  
3                                     The Dark Knight  1704360  2008  
4                                        12 Angry Men   457455  1957  
5                                    Schindler's List   879846  1993  
6                                        Pulp Fiction  1346128  1994  
7       The Lord of the Rings: The Return of the King  1234671  2003  
8                      The Good, the Bad and the Ugly   511481  1966  
9                                          Fight Club  1371330  1999  
10  The Lord of the Rings: The Fellowship of the Ring  1257328  2001  
11     Star Wars: Episode V - The Empire Strikes Back   852316  1980  
12                                       Forrest Gump  1275914  1994  
13                                          Inception  1494900  2010  
14              The Lord of the Rings: The Two Towers  1117984  2002  
15                    One Flew Over the Cuckoo's Nest   690376  1975  
16                                         Goodfellas   740985  1990  
17                                         The Matrix  1237553  1999  
18                                      Seven Samurai   232908  1954  
19                 Star Wars: Episode IV - A New Hope   925378  1977  
20                                        City of God   541304  2002  
21                                              Se7en  1042588  1995  
22                           The Silence of the Lambs   904030  1991  
23                              It's a Wonderful Life   280095  1946  
24                                 The Usual Suspects   753063  1995

## Part 4: Parsing and Exploratory Data Analysis

#### 1. Rename the column headings

#### 2. Run a description of the data

#### 3. Visualize the Data

In [51]:
top25 = pd.read_csv('top25.csv')
top25.drop('Unnamed: 0',axis=1,inplace=True)
top25

ID  Rate                                              Title  \
0   tt0111161   9.3                           The Shawshank Redemption   
1   tt0068646   9.2                                      The Godfather   
2   tt0071562   9.0                             The Godfather: Part II   
3   tt0468569   9.0                                    The Dark Knight   
4   tt0050083   8.9                                       12 Angry Men   
5   tt0108052   8.9                                   Schindler's List   
6   tt0110912   8.9                                       Pulp Fiction   
7   tt0167260   8.9      The Lord of the Rings: The Return of the King   
8   tt0060196   8.9                     The Good, the Bad and the Ugly   
9   tt0137523   8.8                                         Fight Club   
10  tt0120737   8.8  The Lord of the Rings: The Fellowship of the Ring   
11  tt0080684   8.8     Star Wars: Episode V - The Empire Strikes Back   
12  tt0109830   8.8                                       Forrest Gump   
13  tt1375666   8.8                                          Inception   
14  tt0167261   8.7              The Lord of the Rings: The Two Towers   
15  tt0073486   8.7                    One Flew Over the Cuckoo's Nest   
16  tt0099685   8.7                                         Goodfellas   
17  tt0133093   8.7                                         The Matrix   
18  tt0047478   8.7                                      Seven Samurai   
19  tt0076759   8.7                 Star Wars: Episode IV - A New Hope   
20  tt0317248   8.7                                        City of God   
21  tt0114369   8.6                                              Se7en   
22  tt0102926   8.6                           The Silence of the Lambs   
23  tt0038650   8.6                              It's a Wonderful Life   
24  tt0114814   8.6                                 The Usual Suspects   

       Vote  Year  
0   1718172  1994  
1   1174396  1972  
2    804768  1974  
3   1704360  2008  
4    457455  1957  
5    879846  1993  
6   1346128  1994  
7   1234671  2003  
8    511481  1966  
9   1371330  1999  
10  1257328  2001  
11   852316  1980  
12  1275914  1994  
13  1494900  2010  
14  1117984  2002  
15   690376  1975  
16   740985  1990  
17  1237553  1999  
18   232908  1954  
19   925378  1977  
20   541304  2002  
21  1042588  1995  
22   904030  1991  
23   280095  1946  
24   753063  1995

In [52]:
words = pd.read_csv('words.csv')
words.drop('Unnamed: 0',axis=1,inplace=True)
words

Best  Good  Great  Greatest         ID  Perfect
0     29    22     21         5  tt0111161        3
1     20    21     29        15  tt0068646       13
2     19    22     22         6  tt0071562        8
3     31    34     22         1  tt0468569        5
4     13     9     27         5  tt0050083        6
5     13    17      6         4  tt0108052        4
6     15     9      9         4  tt0110912        0
7     27    22     27        11  tt0167260        2
8     31    27     24        10  tt0060196        3
9     19    12     23         2  tt0137523        4
10     8    36     30         9  tt0120737       11
11    24    15     25         7  tt0080684        2
12    18    10     18         2  tt0109830        3
13    12    18     16         2  tt1375666        4
14    17    13     17         6  tt0167261        1
15    17    19     20         8  tt0073486        6
16    23    13     21         7  tt0099685        5
17    13    10     20         0  tt0133093        3
18     6     6     26         5  tt0047478        5
19    16    11     25        12  tt0076759        3
20     7    12      8         1  tt0317248        6
21    18    20     15         6  tt0114369        6
22    20    11      9         2  tt0102926        6
23    14    19     16         4  tt0038650        3
24     9    14     11         8  tt0114814        3

In [53]:
table = top25.merge(words, on = top25.ID)

In [54]:
table

ID_x  Rate                                              Title  \
0   tt0111161   9.3                           The Shawshank Redemption   
1   tt0068646   9.2                                      The Godfather   
2   tt0071562   9.0                             The Godfather: Part II   
3   tt0468569   9.0                                    The Dark Knight   
4   tt0050083   8.9                                       12 Angry Men   
5   tt0108052   8.9                                   Schindler's List   
6   tt0110912   8.9                                       Pulp Fiction   
7   tt0167260   8.9      The Lord of the Rings: The Return of the King   
8   tt0060196   8.9                     The Good, the Bad and the Ugly   
9   tt0137523   8.8                                         Fight Club   
10  tt0120737   8.8  The Lord of the Rings: The Fellowship of the Ring   
11  tt0080684   8.8     Star Wars: Episode V - The Empire Strikes Back   
12  tt0109830   8.8                                       Forrest Gump   
13  tt1375666   8.8                                          Inception   
14  tt0167261   8.7              The Lord of the Rings: The Two Towers   
15  tt0073486   8.7                    One Flew Over the Cuckoo's Nest   
16  tt0099685   8.7                                         Goodfellas   
17  tt0133093   8.7                                         The Matrix   
18  tt0047478   8.7                                      Seven Samurai   
19  tt0076759   8.7                 Star Wars: Episode IV - A New Hope   
20  tt0317248   8.7                                        City of God   
21  tt0114369   8.6                                              Se7en   
22  tt0102926   8.6                           The Silence of the Lambs   
23  tt0038650   8.6                              It's a Wonderful Life   
24  tt0114814   8.6                                 The Usual Suspects   

       Vote  Year  Best  Good  Great  Greatest       ID_y  Perfect  
0   1718172  1994    29    22     21         5  tt0111161        3  
1   1174396  1972    20    21     29        15  tt0068646       13  
2    804768  1974    19    22     22         6  tt0071562        8  
3   1704360  2008    31    34     22         1  tt0468569        5  
4    457455  1957    13     9     27         5  tt0050083        6  
5    879846  1993    13    17      6         4  tt0108052        4  
6   1346128  1994    15     9      9         4  tt0110912        0  
7   1234671  2003    27    22     27        11  tt0167260        2  
8    511481  1966    31    27     24        10  tt0060196        3  
9   1371330  1999    19    12     23         2  tt0137523        4  
10  1257328  2001     8    36     30         9  tt0120737       11  
11   852316  1980    24    15     25         7  tt0080684        2  
12  1275914  1994    18    10     18         2  tt0109830        3  
13  1494900  2010    12    18     16         2  tt1375666        4  
14  1117984  2002    17    13     17         6  tt0167261        1  
15   690376  1975    17    19     20         8  tt0073486        6  
16   740985  1990    23    13     21         7  tt0099685        5  
17  1237553  1999    13    10     20         0  tt0133093        3  
18   232908  1954     6     6     26         5  tt0047478        5  
19   925378  1977    16    11     25        12  tt0076759        3  
20   541304  2002     7    12      8         1  tt0317248        6  
21  1042588  1995    18    20     15         6  tt0114369        6  
22   904030  1991    20    11      9         2  tt0102926        6  
23   280095  1946    14    19     16         4  tt0038650        3  
24   753063  1995     9    14     11         8  tt0114814        3

In [55]:
table.drop('ID_y',axis=1,inplace=True)

In [56]:
table.columns = ['ID','Rate','Title','Vote','Year','Best','Good','Great','Greatest','Perfect']

In [57]:
table

ID  Rate                                              Title  \
0   tt0111161   9.3                           The Shawshank Redemption   
1   tt0068646   9.2                                      The Godfather   
2   tt0071562   9.0                             The Godfather: Part II   
3   tt0468569   9.0                                    The Dark Knight   
4   tt0050083   8.9                                       12 Angry Men   
5   tt0108052   8.9                                   Schindler's List   
6   tt0110912   8.9                                       Pulp Fiction   
7   tt0167260   8.9      The Lord of the Rings: The Return of the King   
8   tt0060196   8.9                     The Good, the Bad and the Ugly   
9   tt0137523   8.8                                         Fight Club   
10  tt0120737   8.8  The Lord of the Rings: The Fellowship of the Ring   
11  tt0080684   8.8     Star Wars: Episode V - The Empire Strikes Back   
12  tt0109830   8.8                                       Forrest Gump   
13  tt1375666   8.8                                          Inception   
14  tt0167261   8.7              The Lord of the Rings: The Two Towers   
15  tt0073486   8.7                    One Flew Over the Cuckoo's Nest   
16  tt0099685   8.7                                         Goodfellas   
17  tt0133093   8.7                                         The Matrix   
18  tt0047478   8.7                                      Seven Samurai   
19  tt0076759   8.7                 Star Wars: Episode IV - A New Hope   
20  tt0317248   8.7                                        City of God   
21  tt0114369   8.6                                              Se7en   
22  tt0102926   8.6                           The Silence of the Lambs   
23  tt0038650   8.6                              It's a Wonderful Life   
24  tt0114814   8.6                                 The Usual Suspects   

       Vote  Year  Best  Good  Great  Greatest  Perfect  
0   1718172  1994    29    22     21         5        3  
1   1174396  1972    20    21     29        15       13  
2    804768  1974    19    22     22         6        8  
3   1704360  2008    31    34     22         1        5  
4    457455  1957    13     9     27         5        6  
5    879846  1993    13    17      6         4        4  
6   1346128  1994    15     9      9         4        0  
7   1234671  2003    27    22     27        11        2  
8    511481  1966    31    27     24        10        3  
9   1371330  1999    19    12     23         2        4  
10  1257328  2001     8    36     30         9       11  
11   852316  1980    24    15     25         7        2  
12  1275914  1994    18    10     18         2        3  
13  1494900  2010    12    18     16         2        4  
14  1117984  2002    17    13     17         6        1  
15   690376  1975    17    19     20         8        6  
16   740985  1990    23    13     21         7        5  
17  1237553  1999    13    10     20         0        3  
18   232908  1954     6     6     26         5        5  
19   925378  1977    16    11     25        12        3  
20   541304  2002     7    12      8         1        6  
21  1042588  1995    18    20     15         6        6  
22   904030  1991    20    11      9         2        6  
23   280095  1946    14    19     16         4        3  
24   753063  1995     9    14     11         8        3

## Part 3: Build the Decision Tree

#### 1. What is our target attribute? 

In [90]:
x = table[[col for col in table.columns if col not in ['ID','Rate','Title','Year']]]
y = table['Rate']

In [93]:
x

Vote  Best  Good  Great  Greatest  Perfect
0   1718172    29    22     21         5        3
1   1174396    20    21     29        15       13
2    804768    19    22     22         6        8
3   1704360    31    34     22         1        5
4    457455    13     9     27         5        6
5    879846    13    17      6         4        4
6   1346128    15     9      9         4        0
7   1234671    27    22     27        11        2
8    511481    31    27     24        10        3
9   1371330    19    12     23         2        4
10  1257328     8    36     30         9       11
11   852316    24    15     25         7        2
12  1275914    18    10     18         2        3
13  1494900    12    18     16         2        4
14  1117984    17    13     17         6        1
15   690376    17    19     20         8        6
16   740985    23    13     21         7        5
17  1237553    13    10     20         0        3
18   232908     6     6     26         5        5
19   925378    16    11     25        12        3
20   541304     7    12      8         1        6
21  1042588    18    20     15         6        6
22   904030    20    11      9         2        6
23   280095    14    19     16         4        3
24   753063     9    14     11         8        3

In [91]:
# Scale X
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(x)

In [92]:
X 

array([[ 1.8475406 ,  1.65525447,  0.6909739 ,  0.22646345, -0.18454293,
        -0.56568542],
       [ 0.48289809,  0.35304379,  0.55601806,  1.41837634,  2.52932366,
         2.96984848],
       [-0.44470829,  0.20835371,  0.6909739 ,  0.37545256,  0.08684373,
         1.20208153],
       [ 1.81287846,  1.94463462,  2.31044396,  0.37545256, -1.27008957,
         0.14142136],
       [-1.31631367, -0.65978675, -1.06345201,  1.12039812, -0.18454293,
         0.49497475],
       [-0.25629499, -0.65978675,  0.0161947 , -2.00837322, -0.45592959,
        -0.21213203],
       [ 0.91387111, -0.37040659, -1.06345201, -1.56140589, -0.45592959,
        -1.6263456 ],
       [ 0.63416226,  1.36587432,  0.6909739 ,  1.12039812,  1.44377703,
        -0.91923882],
       [-1.18073178,  1.94463462,  1.36575309,  0.67343078,  1.17239037,
        -0.56568542],
       [ 0.97711723,  0.20835371, -0.65858449,  0.52444167, -0.99870291,
        -0.21213203],
       [ 0.69102153, -1.38323713,  2.58035564,  1.

In [94]:
X = pd.DataFrame(X,columns=['Vote','Best','Good','Great','Greatest','Perfect'])

In [95]:
X

Vote      Best      Good     Great  Greatest   Perfect
0   1.847541  1.655254  0.690974  0.226463 -0.184543 -0.565685
1   0.482898  0.353044  0.556018  1.418376  2.529324  2.969848
2  -0.444708  0.208354  0.690974  0.375453  0.086844  1.202082
3   1.812878  1.944635  2.310444  0.375453 -1.270090  0.141421
4  -1.316314 -0.659787 -1.063452  1.120398 -0.184543  0.494975
5  -0.256295 -0.659787  0.016195 -2.008373 -0.455930 -0.212132
6   0.913871 -0.370407 -1.063452 -1.561406 -0.455930 -1.626346
7   0.634162  1.365874  0.690974  1.120398  1.443777 -0.919239
8  -1.180732  1.944635  1.365753  0.673431  1.172390 -0.565685
9   0.977117  0.208354 -0.658584  0.524442 -0.998703 -0.212132
10  0.691022 -1.383237  2.580356  1.567365  0.901004  2.262742
11 -0.325383  0.931804 -0.253717  0.822420  0.358230 -0.919239
12  0.737664  0.063664 -0.928496 -0.220504 -0.998703 -0.565685
13  1.287224 -0.804477  0.151151 -0.518482 -0.998703 -0.212132
14  0.341328 -0.081026 -0.523629 -0.369493  0.086844 -1.272792
15 -0.731783 -0.081026  0.286106  0.077474  0.629617  0.494975
16 -0.604776  0.787114 -0.523629  0.226463  0.358230  0.141421
17  0.641395 -0.659787 -0.928496  0.077474 -1.541476 -0.565685
18 -1.879830 -1.672617 -1.468320  0.971409 -0.184543  0.141421
19 -0.142029 -0.225717 -0.793540  0.822420  1.715164 -0.565685
20 -1.105889 -1.527927 -0.658584 -1.710395 -1.270090  0.494975
21  0.152117  0.063664  0.421062 -0.667471  0.086844  0.494975
22 -0.195604  0.353044 -0.793540 -1.561406 -0.998703  0.494975
23 -1.761411 -0.515097  0.286106 -0.518482 -0.455930 -0.565685
24 -0.574465 -1.238547 -0.388673 -1.263428  0.629617 -0.565685

In [96]:
# if we use MSE, more close to 0 more good. if we use r2, more close 1 more good.
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
np.random.seed(1)
cv = KFold(len(y),shuffle=True,n_folds = 5)
def validation_score(model):
    scores = cross_val_score(model, X,y,cv=cv,scoring='mean_squared_error')
    print scores
    print scores.mean()
    

In [97]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor, AdaBoostRegressor

In [98]:
# Compare different model's cross_validation scores

In [99]:
validation_score(DecisionTreeRegressor(random_state=1))

[-0.03  -0.04  -0.054 -0.032 -0.024]
-0.036


In [100]:
validation_score(RandomForestRegressor(random_state=1))

[-0.01526 -0.01784 -0.04766 -0.03578 -0.01642]
-0.026592


In [101]:
BaggingTree = BaggingRegressor(DecisionTreeRegressor(random_state=1))

In [102]:
validation_score(BaggingTree)

[-0.0077  -0.01518 -0.04734 -0.03828 -0.01772]
-0.025244


In [103]:
validation_score(AdaBoostRegressor(random_state=1))

[-0.0255999  -0.04042156 -0.05090215 -0.04283    -0.01459877]
-0.0348704748739


In [104]:
validation_score(GradientBoostingRegressor(random_state=1))

[-0.00506293 -0.02758616 -0.03862168 -0.03614248 -0.02023727]
-0.0255301038704


In [105]:
#BaggingTree, GradientBoostingRegressor,RandomForestRegressor,AdaBoostRegressor,DecisionTreeRegressor 

In [176]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
def evaluate_model(model):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    #print 'done'
    score = model.score(X_test,y_test) 
    print score

In [198]:
#DecisionTreeRegressor-Optimal model
from sklearn.grid_search import GridSearchCV
params = {'max_depth': [2,4,6,8,10,15],
         'min_samples_split': [1,2,3,4],
         'min_samples_leaf': [1,2,3,4,5,6]}
cv = KFold(len(y),n_folds=5,shuffle=True)
tree = DecisionTreeRegressor(random_state=1)
gs_tree = GridSearchCV(tree,params,cv=cv,n_jobs=-1,scoring='mean_squared_error')
gs_tree.fit(X,y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=25, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=1,
           splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'min_samples_split': [1, 2, 3, 4], 'max_depth': [2, 4, 6, 8, 10, 15], 'min_samples_leaf': [1, 2, 3, 4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, scoring='mean_squared_error',
       verbose=0)

In [199]:
gs_tree.best_params_

{'max_depth': 2, 'min_samples_leaf': 6, 'min_samples_split': 1}

In [200]:
gs_tree.best_score_

-0.024816496598639524

In [201]:
gs_tree.best_estimator_

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=6, min_samples_split=1,
           min_weight_fraction_leaf=0.0, presort=False, random_state=1,
           splitter='best')

In [202]:
tree_optimal = gs_tree.best_estimator_

In [204]:
# R2 
evaluate_model(tree_optimal)

0.0909090909091


In [205]:
#RandomForestRegressor - optimal model
params = {'n_estimators': [10,20,30,40],
         'min_samples_split':[1,2,3,4,5],
         'min_samples_leaf':[1,2,3,4,5]}
cv = KFold(len(y),shuffle=True,n_folds=5)
forest = RandomForestRegressor(random_state=1)
gs_forest = GridSearchCV(forest,params,cv=cv,n_jobs=-1,scoring='mean_squared_error')
gs_forest.fit(X,y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=25, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=1,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [10, 20, 30, 40], 'min_samples_split': [1, 2, 3, 4, 5], 'min_samples_leaf': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, scoring='mean_squared_error',
       verbose=0)

In [206]:
gs_forest.best_score_

-0.025925281235267921

In [207]:
gs_forest.best_params_

{'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 30}

In [208]:
gs_forest.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=2,
           min_samples_split=5, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=1, oob_score=False, random_state=1,
           verbose=0, warm_start=False)

In [209]:
forest_optimal = gs_forest.best_estimator_

In [210]:
evaluate_model(forest_optimal)

0.0391172563058


In [214]:
#GradientBoostingRegressor - optimal model
params = {'learning_rate': [0.1,0.2,0.3],
         'n_estimators': [80,100,120,150],
          'min_samples_split':[1,2,3,4,5],
         'min_samples_leaf':[1,2,3,4,5],
         'max_depth':[2,3,4,5]
         }
cv =KFold(len(y),shuffle=True,n_folds=5)
Boosting = GradientBoostingRegressor(random_state=1)
gs_boosting = GridSearchCV(Boosting,params,cv=cv,n_jobs=-1,scoring='mean_squared_error')
gs_boosting.fit(X,y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=25, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=1, subsample=1.0, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [80, 100, 120, 150], 'min_samples_split': [1, 2, 3, 4, 5], 'learning_rate': [0.1, 0.2, 0.3], 'max_depth': [2, 3, 4, 5], 'min_samples_leaf': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, scoring='mean_squared_error',
       verbose=0)

In [215]:
gs_boosting.best_score_

-0.025685456777324225

In [216]:
gs_boosting.best_params_

{'learning_rate': 0.3,
 'max_depth': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 80}

In [217]:
gs_boosting.best_estimator_

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.3, loss='ls',
             max_depth=3, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=5,
             min_weight_fraction_leaf=0.0, n_estimators=80, presort='auto',
             random_state=1, subsample=1.0, verbose=0, warm_start=False)

In [218]:
boosting_optimal = gs_boosting.best_estimator_

In [219]:
evaluate_model(boosting_optimal)

-0.0586859884891


In [ ]:
#BaggingTree

In [222]:
baggingtree = BaggingRegressor(tree_optimal)
evaluate_model(baggingtree)

-0.0343878580596


In [228]:
params = {'n_estimators': [10, 20, 30, 40],
                  'max_samples': [0.3,0.5,0.7,0.8,1.0],
                 }
cv = KFold(len(y),n_folds=5,shuffle=True)

gsbaggingtree = GridSearchCV(baggingtree, params, cv=cv,n_jobs=-1,scoring='mean_squared_error')
gsbaggingtree.fit(X,y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=25, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=BaggingRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=6, min_samples_split=1,
           min_weight_fraction_leaf=0.0, presort=False, random_state=1,
           splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [10, 20, 30, 40], 'max_samples': [0.3, 0.5, 0.7, 0.8, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring='mean_squared_error',
       verbose=0)

In [229]:
gsbaggingtree.best_score_

-0.025166600123252957

In [230]:
gsbaggingtree.best_estimator_

BaggingRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=6, min_samples_split=1,
           min_weight_fraction_leaf=0.0, presort=False, random_state=1,
           splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [231]:
baggingtree_optimal = gsbaggingtree.best_estimator_

In [232]:
evaluate_model(baggingtree_optimal)

0.0310689862341


#### 6. What is overfitting and how are we at risk? 